# Review sentiment analysis

There is a reviews df that has all the reviews of the listed homes. <br>
I want to incorporate this data as well, I will run a sentiment analysis on each review. <br>
Since some places have received multiple reviews, I will then average them to get an average sentiment review for each rental. <br>
As some rentals don't have reviews, I will have some decisions on how best to use this.  

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [4]:
reviews = pd.read_csv('../data/reviews 2.csv')


In [5]:
reviews.isna().sum()

listing_id        0
id                0
date              0
reviewer_id       0
reviewer_name     0
comments         45
dtype: int64

In [6]:
#blank comments arent very helpful...especially given we have over a quarter million reviews
reviews=reviews[reviews['comments'].notna()]

In [7]:
reviews.isna().sum()

listing_id       0
id               0
date             0
reviewer_id      0
reviewer_name    0
comments         0
dtype: int64

In [9]:
reviews.tail(2)

,listing_id,id,date,reviewer_id,reviewer_name,comments
275154,850831095366621257,854362248994822292,2023-03-24,7007892,Sophia,The room was fine and functional for my purpos...
275155,850831095366621257,856583297333057762,2023-03-27,18759800,Danicia,Decent place. Host was responsive and helpful.


In [33]:
sia = SentimentIntensityAnalyzer()


for index, row in reviews.iterrows():

    sentiments = sia.polarity_scores(row['comments'])

    reviews.at[index, 'review_sent_compound'] = sentiments['compound']
    reviews.at[index, 'review_sent_pos'] = sentiments['pos']
    reviews.at[index, 'review_sent_neg'] = sentiments['neg']
    reviews.at[index, 'review_sent_neu'] = sentiments['neu']
    
#chatGPT


In [34]:
reviews.head(3)

,listing_id,id,date,reviewer_id,reviewer_name,comments,review_sent_compound,review_sent_pos,review_sent_neg,review_sent_neu
0,177,99461681,2016-09-04,78348548,Brenna,"Great place to stay! Definitely a tiny home, b...",0.9918,0.519,0.000,0.481
1,177,101460574,2016-09-13,72125554,Rachael,Joe was very friendly and gave us all the info...,0.7419,0.148,0.038,0.814
2,177,101681475,2016-09-14,21300329,Haley,Amazing property and Joe was a great host! Eve...,0.9039,0.383,0.000,0.617


In [35]:
reviews.shape

(275111, 10)

In [36]:
reviews['listing_id'].nunique()

4402

In [37]:
combined= reviews.groupby('listing_id').agg({'comments': ' '.join, 'review_sent_compound': 'mean', 
                                             'review_sent_pos': 'mean', 
                                             'review_sent_neg': 'mean', 'review_sent_neu': 'mean'})

combined.head()

,comments,review_sent_compound,review_sent_pos,review_sent_neg,review_sent_neu
listing_id,,,,,
177,"Great place to stay! Definitely a tiny home, b...",0.822648,0.376125,0.007658,0.616217
360,"This space was perfect! Great location, hosts,...",0.897685,0.392426,0.005500,0.602074
364,The place was great! Everything we wanted and...,0.931729,0.317586,0.009816,0.672632
590,Easy access to downtown via RTD; lighthearted ...,0.869453,0.315384,0.009521,0.673617
592,I really enjoyed my stay at Jill's. It's a rea...,0.888687,0.309365,0.010509,0.680102


In [40]:
combined = combined.reset_index()

In [41]:
#rename column to match the listing column name

combined = combined.rename(columns={'listing_id': 'id'})


In [43]:
combined.head()

,id,comments,review_sent_compound,review_sent_pos,review_sent_neg,review_sent_neu
0,177,"Great place to stay! Definitely a tiny home, b...",0.822648,0.376125,0.007658,0.616217
1,360,"This space was perfect! Great location, hosts,...",0.897685,0.392426,0.005500,0.602074
2,364,The place was great! Everything we wanted and...,0.931729,0.317586,0.009816,0.672632
3,590,Easy access to downtown via RTD; lighthearted ...,0.869453,0.315384,0.009521,0.673617
4,592,I really enjoyed my stay at Jill's. It's a rea...,0.888687,0.309365,0.010509,0.680102


In [51]:
combined.shape

(4402, 6)

In [ ]:
# combined.to_csv('../data/review_sentiment.csv', index=False)

In [52]:
listings = pd.read_csv('../data/1500_sentiment_api.csv')

In [53]:
listings.shape

(5209, 80)

In [54]:
# There are 1151 rentals listed that don't have any reviews.... 
#which is strange because there are 5250 homes listed and 4402 unique ID's that have reviews... so I would expect 848.

len(set(listings['id'].unique()) - set(combined['id'].unique()))



1132

In [55]:
#there are 303 id's that have been reviewed, but aren't included in the listings... which is strange

# But at least number makese sense as 1151-848 is 303

len(set(combined['id'].unique()) - set(listings['id'].unique()))

325

In [56]:
combined.columns

Index(['id', 'comments', 'review_sent_compound', 'review_sent_pos',
       'review_sent_neg', 'review_sent_neu'],
      dtype='object')

In [57]:
merged = listings.merge(combined[['id', 'comments', 'review_sent_compound', 'review_sent_pos',
       'review_sent_neg', 'review_sent_neu']],
                 on='id')


In [58]:
merged.head()

,id,latitude,longitude,price,shared_status,accommodates,accomodation_group,bathrooms,bedrooms,beds_adjusted,neighborhood,city,has_neighborhood_overview,has_reviews,host_in_CO,host_lives_in_neighborhood,host_id,host_name,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_neighbourhood,host_listings_count,host_total_listings_count,host_has_profile_pic,host_identity_verified,has_host_about,years_hosting,years_of_reviews,license_listed,response_time,host_lives_in_neighborhood.1,name,host_about,description,neighborhood_overview,amenities,minimum_nights,maximum_nights,min_stay_group,max_stay_group,has_availability,availability_30,availability_60,availability_90,availability_365,instant_bookable,number_of_reviews,number_of_reviews_ltm,number_of_reviews_l30d,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,reviews_per_month,bike_scores,walk_scores,transit_scores,property_page_links,name_sent_compound,name_sent_pos,name_sent_neg,name_sent_neu,host_sent_compound,host_sent_pos,host_sent_neg,host_sent_neu,description_sent_compound,description_sent_pos,description_sent_neg,description_sent_neu,neighborhood_sent_compound,neighborhood_sent_pos,neighborhood_sent_neg,neighborhood_sent_neu,25*k,comments,review_sent_compound,review_sent_pos,review_sent_neg,review_sent_neu
0,177,39.69551,-104.92489,79.0,entire home,2,0-2,1.0,1.0,1.0,Virginia Village,Denver,1,1,1,1,615,Joe,within an hour,1.0,1.00,1.0,Virginia Village,2.0,2.0,1.0,0.0,1,15.0,6.0,1,0.0,1,Tiny Home in the Heart of the City- ECO FRIENDLY,"I'm originally from Oklahoma, but have lived i...","160 sq ft + 80 sq ft loft for sleeping, Sleeps...","Quiet neighborhood next to park, creeks and bi...","[""Hot water"", ""Long term stays allowed"", ""Coff...",29,400,month,long_term,1,29,59,89,364,0,120,24,0,4.85,4.96,4.89,4.97,4.94,4.85,4.78,1.56,84.0,51.0,37.0,https://www.walkscore.com/score/loc/lat=39.695...,0.6037,0.304,0.0,0.696,0.6948,0.192,0.000,0.808,0.9476,0.153,0.069,0.778,0.0000,0.000,0.000,1.000,4.0,"Great place to stay! Definitely a tiny home, b...",0.822648,0.376125,0.007658,0.616217
1,360,39.76758,-105.00316,133.0,entire home,3,3-4,1.0,2.0,2.0,Highland,Denver,1,1,1,1,666,Jennifer & Giovanni,within an hour,1.0,0.87,1.0,Highland,4.0,4.0,1.0,1.0,1,15.0,4.0,1,0.0,1,Sit in the Peaceful Garden of the Chickadee Co...,We are artists and tinkerers.\r\n \r\nWe enjoy...,Enjoy the famous Colorado weather and unplug i...,The cottage is located in the center of Lower ...,"[""Coffee maker"", ""Washer"", ""Bedroom comforts"",...",29,35,month,2_months,1,0,0,30,30,0,174,7,1,4.99,4.99,4.96,5.00,5.00,5.00,4.91,3.26,93.0,90.0,49.0,https://www.walkscore.com/score/loc/lat=39.767...,0.4939,0.242,0.0,0.758,0.9694,0.141,0.023,0.835,0.9814,0.271,0.019,0.709,0.1531,0.025,0.018,0.956,6.0,"This space was perfect! Great location, hosts,...",0.897685,0.392426,0.005500,0.602074
2,2361919,39.77960,-105.03859,76.0,entire home,2,0-2,1.0,1.0,1.0,Berkeley,Denver,1,1,1,1,5811115,Annette,within a few hours,1.0,0.81,1.0,Berkeley,5.0,6.0,1.0,0.0,1,10.0,8.0,0,1.0,1,The Cloisters at Berkeley,"I love people of all ages and types, dogs, lon...",A private studio apartment in an unbeatable ne...,"Boutiques, bars, restaurants galore! Highlands...","[""Hot water"", ""Long term stays allowed"", ""Coff...",30,1125,month,long_term,1,24,54,84,359,0,43,3,0,4.67,4.74,4.56,4.67,4.60,4.93,4.60,0.40,76.0,81.0,34.0,https://www.walkscore.com/score/loc/lat=39.779...,0.0000,0.000,0.0,1.000,0.9769,0.278,0.000,0.722,0.9269,0.150,0.000,0.850,0.4333,0.067,0.026,0.907,4.0,Annette was very nice and she had a super cosy...,0.865804,0.293378,0.015956,0.690667
3,2386289,39.76777,-105.03529,150.0,entire home,4,3-4,1.0,1.0,1.0,West Highland,Denver,1,1,1,1,184771,Melanie,NaN,NaN,1.00,0.0,West Highland,1.0,2.0,1.0,1.0,1,13.0,8.0,1,4.0,1,Sunny Sky Loft in Highlands,"Things I love: dogs, urban gardening, yoga, ho...",W

In [60]:
merged.shape

(4077, 85)

In [1]:
merged.isna().sum()

NameError: name 'merged' is not defined

In [62]:
# merged.to_csv('../data/1500_review_sentiment.csv', index=False)